# Data-Cleaning Code

# imports

In [ ]:
!pip install langdetect
!pip install -U textblob
!pip install -U textblob-fr
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from langdetect import DetectorFactory, detect, detect_langs
from bs4 import BeautifulSoup
import re
import datetime as datetime
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer

text = u"Quelle belle matinée"
blob = TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [ ]:
df = pd.read_csv('/content/Barkha_E2.csv')
df=df.set_axis([*df.columns[:-1], 'Reply'], axis=1, inplace=False)

In [ ]:
df.sample(5)

,Name,Comment,Likes,Reply_Count,Time,date,hour,Reply
110,Kuruangha Yamii,ill definitely fall in love with ayush if i we...,0,0,2020-11-06T19:34:27Z,6,19,11
250,roopesh Raj,barkha mam and ayush sir you both look very cu...,0,0,2020-11-05T10:51:30Z,5,10,11
307,Garima Parajuli,ayush and barkha onscreen chemistry is fire,2,1,2020-11-05T05:29:18Z,5,5,11
37,Robin Lalwani,aayush and barkha you look soooo good together...,0,0,2020-12-17T19:42:57Z,17,19,12
134,Soumya,can someone tell me the name of the guy who is...,1,1,2020-11-06T09:37:54Z,6,9,11


# preprocess

In [ ]:
df_copy = df.copy()
df_copy['Comment'] = [BeautifulSoup(text).get_text() for text in df_copy['Comment']]#remove HTML tags
df_copy['Comment'] = df_copy['Comment'].apply(lambda x: re.split(' "https:\/\/.*', str(x))[0])#remove HTTP

##### Clean data creation 

In [ ]:


# lists for the data columns 
Name=[]
Comment=[]
Time=[]
Likes=[]
Reply=[]
Time=[]
# list for checking error in detect Library
done=[]
Notedone=[]
#loop for the new data
for i in range(len(df_copy)):
    try:
        if detect(df_copy['Comment'][i]) == 'en':
            Comment.append(df_copy['Comment'][i])
            done.append(i)#check
            #others appended
            Name.append(df_copy['Name'][i])
            Time.append(df_copy['Time'][i])
            Likes.append(df_copy['Likes'][i])
            Reply.append(df_copy['Reply'][i])
            Time.append(df_copy['Time'][i])
    except:
        Notedone.append(i)#check

In [ ]:
print(len(Name),len(Comment),len(Time),len(Likes),len(Reply))

531 531 1062 531 531


In [ ]:
print(len(Notedone),'text languages is not getting detected and gave and error')

0 text languages is not getting detected and gave and error


In [ ]:
df_cl = pd.DataFrame(list(zip(Name,Comment,Time,Likes,Reply)),columns =['Names','Comment','Time','Likes','Reply_count'])
#df_cl.to_csv('clean_data.csv',index=False)#save
#df_cl

In [ ]:
temp= pd.DataFrame(list(Comment),columns=['Comment'])
temp1= pd.DataFrame(list(Likes),columns=['Likes'])
temp2= pd.DataFrame(list(Name),columns=['Name'])
temp3= pd.DataFrame(list(Reply),columns=['Reply_Count'])
temp4 = pd.DataFrame(list(Time),columns=['Time'])

In [ ]:
temp['Comment'] = temp.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [ ]:
#removing special characters from comments
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    temp['Comment'] = temp['Comment'].str.replace(char, ' ')

In [ ]:
temp['Comment'] = temp['Comment'].str.split().str.join(" ")

In [ ]:
#removing emojis
temp2['Name'] = temp2.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [ ]:
#removing special characters from name
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    temp2['Name'] = temp2['Name'].str.replace(char, ' ')

### Concating data

In [ ]:
result = pd.concat([temp2, temp], axis=1, join='inner')

In [ ]:
result = pd.concat([result, temp1], axis=1, join='inner')

In [ ]:
result = pd.concat([result, temp3], axis=1, join='inner')

In [ ]:
result = pd.concat([result, temp4], axis=1, join='inner')

In [ ]:
from datetime import *

In [ ]:
#Extracting month, date and hour from time column
date = []
hour = []
month = []
for i in range(len(result)):
    d1 = datetime.fromisoformat(result['Time'][i][:-1])
    h = d1.strftime('%H')
    d = d1.strftime('%d')
    m = d1.strftime('%m')
    date.append(d)
    hour.append(h)
    month.append(m)

result['date']=date
result['hour']=hour
result['month']=month

In [ ]:
#lowering the strings
result['Comment'] = result['Comment'].str.lower()

In [ ]:
result.head()

,Name,Comment,Likes,Reply_Count,Time,date,hour,month
0,Dice Media,make your mornings exciting healthy with so go...,2907,285,2020-11-24T08:37:17Z,24,08,11
1,Squad x 69 Esports,will wait for season 3 barkha singh and ayush ...,1,0,2020-11-24T08:37:17Z,24,08,11
2,Brainable Brains,such a great series i enjoyed watching from be...,1,0,2021-09-15T17:28:23Z,15,17,09
3,Atul Sen,waiting for 3rd session,0,0,2021-09-15T17:28:23Z,15,17,09
4,Sijal Soni,i am so glad to watch this wed session and all...,0,0,2021-09-14T20:37:38Z,14,20,09


In [ ]:
result.to_csv('PFA S2 E6_cleaned.csv', index=False)

# Sentimental Scoring methods

# Importing libraries

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
#tokenizing the comments
df['Tokenize']= df['Comment'].apply(word_tokenize)

In [ ]:
#creating stem of comments
stemmer = SnowballStemmer("english")

In [ ]:
df['stemmed'] = df['Tokenize'].apply(lambda x: [stemmer.stem(y) for y in x])

In [ ]:
#Creating sentiment scores positive, negative, neutral and compound
analyzer = SentimentIntensityAnalyzer()
df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['Comment']]
df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df['Comment']]
df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df['Comment']]
df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df['Comment']]

In [ ]:
#Calculating polarity and subjectivity
df[['polarity', 'subjectivity']] = df['Comment'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

### Empath score 


In [ ]:
!pip install empath

In [ ]:
#function for creating dictionary
def returnSum(dict): 
    sum = 0
    for i in dict: 
        sum = sum + dict[i]  
    return sum

In [ ]:
#calculating empath score
from empath import Empath
lexicon = Empath()
l = []
m = []
for i in range(len(df)):
  lex= lexicon.analyze(df['Comment'][i], normalize=True)
  emp = empath=lexicon.analyze(df['Comment'][i], normalize=True)
  empath_score= returnSum(empath) 
  l.append(empath_score)

In [ ]:
l = pd.DataFrame(list(l))

In [ ]:
l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       206 non-null    float64
dtypes: float64(1)
memory usage: 1.7 KB


In [ ]:
l.rename(columns = {0:'empath_score'}, inplace = True)

In [ ]:
df = pd.concat([df, l], axis=1, join='inner')
#display(df)

,Name,Comment,Likes,Reply_Count,Time,date,hour,month,Tokenize,stemmed,compound,neg,neu,pos,polarity,subjectivity,empath_score
0,CR MOTIONS,here s a teaser of ayush mehra and barkha sing...,0,0,2021-08-29T12:40:33Z,29,12,8,"[here, s, a, teaser, of, ayush, mehra, and, ba...","[here, s, a, teaser, of, ayush, mehra, and, ba...",0.4215,0.081,0.726,0.194,0.500000,0.888889,0.038462
1,Rishika Chakraborty,barkha and aayush are looking so perfect together,6,0,2021-08-09T12:58:28Z,9,12,8,"[barkha, and, aayush, are, looking, so, perfec...","[barkha, and, aayush, are, look, so, perfect, ...",0.6948,0.000,0.596,0.404,1.000000,1.000000,0.500000
2,Aman Sharma,put the entire show aside the long strand of h...,2,0,2021-04-02T15:02:46Z,2,15,4,"[put, the, entire, show, aside, the, long, str...","[put, the, entir, show, asid, the, long, stran...",0.4404,0.000,0.879,0.121,0.162500,0.481250,0.375000
3,Abdul Moiz,barkha is beautiful and ayush is handsomewhat ...,1,0,2021-04-01T22:19:01Z,1,22,4,"[barkha, is, beautiful, and, ayush, is, handso...","[barkha, is, beauti, and, ayush, is, handsomew...",0.8402,0.000,0.529,0.471,0.825000,0.875000,0.416667
4,usic Addiction,barkha and ayush look s super cute as they are...,0,0,2021-03-29T00:50:22Z,29,0,3,"[barkha, and, ayush, look, s, super, cute, as,...","[barkha, and, ayush, look, s, super, cute, as,...",0.9719,0.000,0.395,0.605,0.568056,0.540278,0.480000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Avani Savadatti,i am very exited to see this viedo yay i am bi...,0,0,2020-11-09T12:51:54Z,9,12,11,"[i, am, very, exited, to, see, this, viedo, ya...","[i, am, veri, exit, to, see, this, viedo, yay,...",0.6908,0.000,0.695,0.305,0.200000,0.300000,0.058824
202,Khushi,am sooo damn excited for this i waited for thi...,2,0,2020-11-09T12:45:26Z,9,12,11,"[am, sooo, damn, excited, for, this, i, waited...","[am, sooo, damn, excit, for, this, i, wait, fo...",0.6249,0.085,0.751,0.164,0.231481,0.440741,0.200000
203,Bhabani Kalita,ayush and barkha is love,1,0,2020-11-09T12:39:54Z,9,12,11,"[ayush, and, barkha, is, love]","[ayush, and, barkha, is, love]",0.6369,0.000,0.488,0.512,0.500000,0.600000,1.000000
204,Palak Negi,barkha op ayush op dice media op,1,0,2020-11-09T12:39:54Z,9,12,11,"[barkha, op, ayush, op, dice, media, op]","[barkha, op, ayush, op, dice, media, op]",0.0000,0.000,1.000,0.000,0.000000,0.000000,0.000000


In [ ]:
def maximum(a, b, c):
  
    if (a >= b) and (a >= c):
        largest = 'neg'
  
    elif (b >= a) and (b >= c):
        largest = 'neu'
    else:
        largest = 'pos'
          
    return largest

In [ ]:
l=[]
for i in range(len(result)):
    x=maximum(df['neg'][i],df['neu'][i],df['pos'][i])
    l.append(x)

df['sentiment_status']=l

In [ ]:
#Final dataset cleaned and scored
df.to_csv("Barkha E3 score.csv", index=False)